In [ ]:
import vice
import arya
import matplotlib.pyplot as plt
import pandas as pd
from surp import ViceModel
import numpy as np

## Initial versus final properties

In [ ]:
walks_raw = pd.read_csv("../../models/fiducial/run/milkyway_star_migration.dat")

In [ ]:
walks_if = pd.read_csv("../../models/fiducial/run/migration_initial_final.dat") # requires a bit of memory
walks_if

In [ ]:
t_end = np.max(walks_if.t)

In [ ]:
walks_if["dR"] = walks_if.R_final - walks_if.R_birth
walks_if["dt"] = t_end - walks_if.t

In [ ]:

plt.hist(walks_if.dR / walks_if.dt ** 0.3, 
         density=True,
         range=(-10, 10),
        )

plt.xlabel("delta R")
plt.ylabel("density")

In [ ]:

plt.hist(walks_if.z_final / walks_if.dt ** 0.3, 
         density=True,
         range=(-3, 3),
        )

plt.xlabel("delta z")
plt.ylabel("density")

In [ ]:
t_end = np.max(walks_if.t)

In [ ]:
t_end

## Checking agains full migration

In [ ]:
walks_raw

In [ ]:
walks = walks_raw.set_index(["zone", "time_int"]) # multiindex makes this much easier...
walks.sort_index(inplace=True)

In [ ]:
df_end = walks[walks.time == np.max(walks.time)]

In [ ]:
df_end

In [ ]:
Ns = df_end.index.values # last slice contains every value

In [ ]:
t_int_end = np.max(walks_raw.time_int)

In [ ]:
walks.loc[(1, 250)]

In [ ]:
walks_raw

In [ ]:
t_end = np.max(walks.time)

In [ ]:
Np = len(Ns)
t_birth = np.empty(Np)
Ri = np.zeros(Np)
Rf = np.zeros(Np)
zf = np.zeros(Np)


for n, idx in enumerate(Ns):
    print(f"{n} / {Np}", end="\r")
    ws = walks.loc[idx[0], idx[1]]
    t_birth[n] = ws.time.iloc[0]
    assert ws.time.iloc[-1] == t_end
    Ri[n] = ws.R.iloc[0]
    Rf[n] = ws.R.iloc[-1]
    zf[n] = ws.z.iloc[-1]
    
dR = Rf - Ri

In [ ]:
σ8 = 2.68

In [ ]:
for t in np.arange(0, 13.0):
    dt = 0.5
    filt = t_birth > t - dt
    filt &= t_birth < t + dt
    filt &= Ri > 7
    filt &= Ri < 9
    plt.hist(dR[filt]/np.sqrt(13.2 - t_birth[filt]), 30, histtype="step", density=True, alpha=0.3)
    
x = np.linspace(-4, 4, 1000)
σ = σ8 / np.cbrt(8)
y = 1/np.sqrt(2*np.pi) * 1/σ * np.exp(-x**2 / (2*σ**2))
plt.plot(x, y)
plt.xlabel(r"$dR / \sqrt[3]{t_{\rm  end} - t}$")
plt.ylabel("density")

In [ ]:
dz = np.abs(zf)

In [ ]:
for t in np.arange(0, 13.0):
    dt = 0.5
    filt = t_birth > t - dt
    filt &= t_birth < t + dt
    filt &= Rf > 7
    filt &= Rf < 9
    delta_t = 13.2 - t
    plt.hist(zf[filt] /delta_t**0.33, 30, range=(-2, 2), histtype="step", density=True, alpha=0.3)

x_m = np.linspace(-2, 2, 1000)
hz = 0.5
plt.plot(x_m, 1 /hz / np.cosh(x/hz)**2)
plt.ylabel("density")
plt.xlim(-2, 2)

all time bins appear to be well approximated by the gaussian (yay). The overabundance at 0 is likely due to boundry conditions (r=0 and r=20).

In [ ]:
plt.scatter(13.2 - t_birth, np.abs(dR), s=0.05, alpha=0.1)
plt.xlabel("tend - t birth")
plt.ylabel('dr')

In [ ]:
z0 = 2
dz = 0.1
filt = Ri > z0 - dz
filt &= Ri <= z0 + dz

In [ ]:
idx = Ns[filt]

In [ ]:
for N in idx:
    w = walks.loc[N[0], N[1]]
    plt.plot(w.time, w.R, lw=0.2, alpha=0.1, color="k")
    


In [ ]:
for N in idx:
    w = walks.loc[N[0], N[1]]
    plt.plot(w.time, w.z, lw=0.2, alpha=0.1, color="k")

plt.xlabel("time / Gyr")
plt.ylabel("z / kpc")
